In [1]:
import copy
import folium
import numpy as np
import pandas as pd
import folium.plugins as plugins
from sklearn import preprocessing


In [2]:

def getData():

    # read csv file
    data = pd.read_csv("./fire_archive_M6_96619.csv")
    
    # we will min-max scale brightenss (temperature) field 
    # of data to use it as color gradient in our map
    brightness = data[['brightness']].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    brightness_scaled = min_max_scaler.fit_transform(brightness)
    brightness_scaled = pd.DataFrame(brightness_scaled)

    data['brightness_scaled'] = brightness_scaled
    
    return data


In [3]:

def getCoOrdinates(data):
    
    # get all the dates in dataset
    all_dates = data.acq_date.unique()

    # list to hold all location coordinates that have fire
    coordinates = []
    
    # list to hold all the coordinates that have fire on a given day
    locations_per_day = []

    # for each day get locations that have fire
    for date in range(len(all_dates)):
        
        data_per_day = data[data["acq_date"] == all_dates[date]]
        
        for lat, long, temp in zip(data_per_day['latitude'], data_per_day['longitude'], data_per_day['brightness_scaled']):
            locations_per_day.append([lat, long, temp])
        
        temp = copy.deepcopy(locations_per_day)
        coordinates.append(temp)

    return coordinates


In [4]:

# get dataset
data = getData()
data.head()


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,brightness_scaled
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0,0.063601
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0,0.094423
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0,0.056751
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0,0.049413
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0,0.050391


In [5]:

# get coordinates of all fire hotspots
coordinates = getCoOrdinates(data)


In [6]:

# plot thoose coordinates according to timeline
aus_map = folium.Map([-23., 133.], zoom_start=4.5)
fire_growth = plugins.HeatMapWithTime(coordinates, radius = 10)
fire_growth.add_to(aus_map)

aus_map


In [7]:


# save for future usage
aus_map.save('aus_map.html')
